http://victorlin.me/posts/2012/08/26/good-logging-practice-in-python

| Level      | When it’s used                                               |
| ---------- | ------------------------------------------------------------ |
| `DEBUG`    | Detailed information, typically of interest only when diagnosing problems. |
| `INFO`     | Confirmation that things are working as expected.            |
| `WARNING`  | An indication that something unexpected happened, or indicative of some problem in the near future (e.g. ‘disk space low’). The software is still working as expected. |
| `ERROR`    | Due to a more serious problem, the software has not been able to perform some function. |
| `CRITICAL` | A serious error, indicating that the program itself may be unable to continue running. |

> "Notice that the debug() and info() messages didn’t get logged. This is because, by default, the logging module logs the messages with a severity level of WARNING or above. You can change that by configuring the logging module to log events of all levels if you want. You can also define your own severity levels by changing configurations, but it is generally not recommended as it can cause confusion with logs of some third-party libraries that you might be using." -- https://realpython.com/python-logging/

`--log=INFO`: set the logging level from a command-line option

In [1]:
import logging

In [2]:
log_format = '[%(asctime)s][%(name)s][%(levelname)s] %(message)s'

In [3]:
(logging.DEBUG, 
logging.INFO, 
logging.WARNING, 
logging.ERROR, 
logging.CRITICAL)

(10, 20, 30, 40, 50)

In [4]:
logging.NOTSET

0

# Root Logger

In [6]:
logging.basicConfig(format=log_format, 
                    datefmt="%Y-%m-%dT%H:%M:%S%z", # ISO format
                    level=logging.DEBUG, 
                    
                    # filename='log', 
                    # encoding='utf-8', 
                    # filemode='w'
                    )

In [7]:
loglevel = 'DEBUG'
getattr(logging, loglevel.upper())

10

In [8]:
logging.debug('debug')
logging.info('info')
logging.warning('warning')
logging.error('error')
logging.critical('critical')

[2022-03-15T09:44:44+0000][root][DEBUG] debug
[2022-03-15T09:44:44+0000][root][INFO] info
[2022-03-15T09:44:44+0000][root][WARNING] warning
[2022-03-15T09:44:44+0000][root][ERROR] error
[2022-03-15T09:44:44+0000][root][CRITICAL] critical


# Custom Logger

https://docs.python.org/3/howto/logging-cookbook.html

In [ ]:
# name = __name__
name = 'logger_00'
logger = logging.getLogger(name)

logger.setLevel(logging.DEBUG)
# logger.setFormatter('[%(asctime)s][%(levelname)s] %(message)s')

In [ ]:
# create file handler
fh = logging.FileHandler(filename='log', 
                         encoding='utf-8', 
                         mode='w')
fh.setLevel(logging.DEBUG) # The logger's level filters every message before it can reach its handlers

# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter(log_format)
fh.setFormatter(formatter)
ch.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [ ]:
logger.debug('debug')
logger.info('info')
logger.warning('warning')
logger.error('error')
logger.critical('critical')

# LoggerAdapter()

In [ ]:
# https://stackoverflow.com/questions/17558552/how-do-i-add-custom-field-to-python-log-format-string

logger = logging.getLogger(__name__)
syslog = logging.StreamHandler()
formatter = logging.Formatter('[%(asctime)s][%(app_name)s] %(message)s')
syslog.setFormatter(formatter)
logger.setLevel(logging.INFO)
logger.addHandler(syslog)

# class extra_(object):
#     def __init__(self):
#         self.items = { 'app_name': 'App Name' }
#     def __getitem__(self, key):
#         return self.items[key]
#     def __setitem__(self, key, value):
#         self.items[key] = value
# extra = extra_()
# extra['app_name'] = 'App Name2'
extra = { 'app_name': 'App Name' }
logger = logging.LoggerAdapter(logger, extra)
logger.info('info')

# extra = { 'app_name': 'New Name' }
# logger.info('info')

# AppFilter()

In [3]:
# https://stackoverflow.com/questions/17558552/how-do-i-add-custom-field-to-python-log-format-string

class FilterClass(logging.Filter):
    def __init__(self):
        self.app_name = 'App Name'
        self.value    = -1
    def filter(self, record):
        record.app_name = self.app_name
        record.value = self.value
        return True

logger = logging.getLogger(__name__)

app_filter = FilterClass()
logger.addFilter(app_filter)

syslog = logging.StreamHandler()

formatter = logging.Formatter('[%(asctime)s][%(app_name)s][%(value)s] %(message)s')
syslog.setFormatter(formatter)

logger.setLevel(logging.INFO)
logger.addHandler(syslog)

logger.info('info')

app_filter.app_name = 'New Name'
app_filter.value = 99
logger.info('info')

[2022-03-14 15:00:20,034][App Name][-1] info
[2022-03-14 15:00:20,034][App Name][-1] info
[2022-03-14 15:00:20,036][New Name][99] info
[2022-03-14 15:00:20,036][New Name][99] info
